In [2]:
import numpy as np
import os, sys
sys.path.append('../')
from src.datasets.soccernet_generic import soccernet_dataset_generic
from src.utils.helper import samples_by_language
import torch
from torch.utils.data import Dataset, DataLoader

/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
root_dir = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/"
train_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Train_300.npy"
valid_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Valid_100.npy"
test_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Test_100.npy"


In [4]:
%%bash
which python3

/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/bin/python3


In [5]:
%%bash
nvidia-smi

Tue Nov 24 11:37:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.165.02   Driver Version: 418.165.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:36:00.0 Off |                    0 |
| N/A   31C    P0    58W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [8]:
langpath_train = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/train_lang_dict.json'
langpath_valid = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/valid_lang_dict.json'
langpath_test = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/test_lang_dict.json'

samples_train = samples_by_language(langpath_train,train_list,'all')
samples_valid = samples_by_language(langpath_valid,valid_list,'all')
samples_test = samples_by_language(langpath_test,test_list,'all')


<function all(iterable, /)>

In [9]:
train_set = soccernet_dataset_generic(npy_file=train_list,root_dir=root_dir,lang='all',lang_dict=langpath_train)
valid_set = soccernet_dataset_generic(npy_file=valid_list,root_dir=root_dir,lang='all',lang_dict=langpath_valid)


In [10]:
train_set.load_waves()
valid_set.load_waves()


In [11]:
train_set.generate_mel_spectrograms()
valid_set.generate_mel_spectrograms()


/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/lib/python3.6/site-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  "At least one mel filterbank has all zero values. "
/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/lib/python3.6/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/lib/python3.6/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed i

In [12]:
train_set.load_resnet_features()
valid_set.load_resnet_features()


# Train on resnet features

In [120]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self,window_size):
        super(Net, self).__init__()
        self.window_size = window_size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=128, kernel_size=(512,1))
        self.bn1 = nn.BatchNorm2d(self.conv1.out_channels)
        self.conv2 = nn.Conv2d(self.conv1.out_channels, 16, 1)
        self.bn2 = nn.BatchNorm2d(self.conv2.out_channels)
        self.fc1 = nn.Linear(16*8, 120)
        self.fc2 = nn.Linear(self.fc1.out_features, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = x.view(-1,16*8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [121]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net(10)
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.025, momentum=0.9)

In [122]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

params = {'batch_size': 12,
         'shuffle': True,
         'num_workers':4,
         'drop_last':True}

In [123]:
trainloader = DataLoader(train_set,**params)
validloader = DataLoader(valid_set,**params)

In [124]:
net.to(device)
epochs = 2
accs = list()
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        inputs = data['resnet_spot'].unsqueeze(1).permute(0,1,3,2).to(device)
        target = data['label'].to(device)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

            if i % 5 == 0:    # print every 2000 mini-batches

                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 5))
                running_loss = 0.0

    # calculate accuracy
    with torch.no_grad():
        net.eval()
        res = torch.zeros((3,3))
        for i, data in enumerate(validloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['resnet_spot'].unsqueeze(1).permute(0,1,3,2).to(device)
            label = data['label'].to(device)

            # forward + backward + optimize
            outputs = net(inputs)

            preds = torch.argmax(outputs,dim=1)

            for p,gt in zip(preds,label):
                res[int(p),int(gt)] += 1




        N_total = res.sum()
        N_correct = res.diag().sum()

        acc = N_correct / N_total

        print(f"Epoch : {epoch}, Accuracy : {acc}")
        accs.append(acc)

        print('Finished Training')


Epoch : 0, Accuracy : 0.927370011806488
Finished Training
Epoch : 1, Accuracy : 0.9243119359016418
Finished Training


# Train on audio

In [118]:
import torchvision
densenet = torchvision.models.densenet161(pretrained=True)
densenet.features.conv0 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
densenet.classifier = nn.Linear(in_features=densenet.classifier.in_features, out_features=3,bias=True)
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet.parameters(), lr=0.0001, momentum=0.9)


In [136]:
densenet.to(device)
epochs = 2
accs_ms = list()
for epoch in range(epochs):
    for i, data in enumerate(trainloader,0):
        
        densenet.train()
        inputs = data['ms_spot'].unsqueeze(1).permute(0,1,2,3).to(device)
        target = data['label'].to(device)
        inputs[inputs.isnan()] = 0.0
        optimizer.zero_grad()


        # forward + backward + optimize
        outputs = densenet(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            running_loss += loss.item()
            # print statistics


    # calculate accuracy
    with torch.no_grad():
        densenet.eval()
        res = torch.zeros((3,3))
        for i, data in enumerate(validloader
                                 , 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['ms_spot'].unsqueeze(1).permute(0,1,2,3).to(device)
            label = data['label'].to(device)

            # forward + backward + optimize
            outputs = densenet(inputs)

            preds = torch.argmax(outputs,dim=1)

            for p,gt in zip(preds,label):
                res[int(p),int(gt)] += 1




        N_total = res.sum()
        N_correct = res.diag().sum()

        acc = N_correct / N_total
        accs_ms.append(acc)
        print(f"Epoch : {epoch}, Accuracy : {acc}")


print('Finished Training')


Epoch : 0, Accuracy : 0.78899085521698
Epoch : 1, Accuracy : 0.783639132976532
Finished Training


# Merge models 1 - softmax average during eval

In [154]:
# calculate accuracy


with torch.no_grad():
    densenet.eval()
    net.eval()
    
    res_visual = torch.zeros((3,3))
    res_audio = torch.zeros((3,3))
    res = torch.zeros((3,3))
    
    
    for i, data in enumerate(validloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs_audio = data['ms_spot'].unsqueeze(1).permute(0,1,2,3).to(device)
        inputs_visual = data['resnet_spot'].unsqueeze(1).permute(0,1,3,2).to(device)
        label = data['label'].to(device)
        
        inputs_audio[inputs_audio.isnan()] = 0.0

        # forward + backward + optimize
        outputs_audio = densenet(inputs_audio)
        outputs_visual = net(inputs_visual)

        fused_preds = torch.softmax(outputs_audio,dim=1) + torch.softmax(outputs_visual,dim=1)
        preds_audio = torch.argmax(outputs_audio,dim=1)
        preds_visual = torch.argmax(outputs_visual,dim=1)
        
        preds = torch.argmax(fused_preds,dim=1)
        for p,gt in zip(preds,label):
            res[int(p),int(gt)] += 1
        
        for p,gt in zip(preds_audio,label):
            res_audio[int(p),int(gt)] += 1
            
        for p,gt in zip(preds_visual,label):
            res_visual[int(p),int(gt)] += 1




    N_total = res.sum()
    N_correct = res.diag().sum()

    acc = N_correct / N_total
    accs_ms.append(acc)
    print(f"Epoch : {epoch}, Accuracy : {acc}")


Epoch : 1, Accuracy : 0.9403669834136963


In [165]:
def get_acc(res_a):
    N_total = res_a.sum()
    N_correct = res_a.diag().sum()

    acc = N_correct / N_total
    print(acc)

In [169]:
get_acc(res)

tensor(0.9404)


In [162]:
res_visual

tensor([[349.,  39.,  14.],
        [ 44., 515.,   0.],
        [  2.,   7., 338.]])

# Merge models 2 - layers and training

In [170]:
A = net
B = densenet


In [176]:
A.fc3 

Linear(in_features=120, out_features=84, bias=True)

In [174]:
B.classifier 

2208

In [249]:
import torch.nn as nn
import torch.nn.functional as F


class fusion(nn.Module):
    def __init__(self):
        super(fusion, self).__init__()
        self.fc1 = nn.Linear(256,128)
        self.fc2 = nn.Linear(128,3)
        self.bn = nn.BatchNorm1d(256)

    def forward(self, x_audio,x_visual):
        x = torch.cat((x_audio,x_visual),dim=1)
        print(x.size())
        x = x.view(-1,256)
        x = self.bn(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x

In [236]:
densenet.eval()
net.eval()


Net(
  (conv1): Conv2d(1, 128, kernel_size=(512, 1), stride=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=128, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=128, bias=True)
)

In [237]:
densenet.classifier = nn.Linear(2208,out_features=128,bias=True)
net.fc3 = nn.Linear(84,128,bias=True)

In [250]:
f = fusion()

In [251]:
net.to(device)
densenet.to(device)
f.to(device)

fusion(
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [252]:
with torch.no_grad():
    audio = densenet(inputs_audio)
    visual = net(inputs_visual)

In [253]:
audio

tensor([[ 0.3135, -0.7256, -0.3348,  ..., -0.0286, -0.3306,  0.1435],
        [-0.0595, -0.4561, -0.0425,  ..., -0.3128,  0.2838, -0.4069],
        [ 0.2491, -0.6167, -0.3473,  ...,  0.3357,  0.4257, -0.1191],
        ...,
        [ 0.4718, -0.5160,  0.1843,  ..., -0.3095,  0.0480,  0.0574],
        [ 0.1466, -0.2967, -0.0260,  ..., -0.0955,  0.6621, -0.5232],
        [-0.2893, -0.4822, -0.2177,  ...,  0.3221,  0.0219, -0.0936]],
       device='cuda:0')

In [255]:
f(audio,visual)

torch.Size([12, 256])


tensor([[0.0000, 0.1746, 0.1349],
        [0.3369, 0.0000, 0.0000],
        [0.1472, 0.0000, 0.0000],
        [0.0794, 0.0000, 0.0293],
        [0.0362, 0.0000, 0.1800],
        [0.1757, 0.4450, 0.0000],
        [0.3192, 0.0257, 0.0000],
        [0.1620, 0.0993, 0.0000],
        [0.1398, 0.1918, 0.0000],
        [0.0337, 0.4818, 0.0000],
        [0.3250, 0.0000, 0.0000],
        [0.0220, 0.1986, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(f.parameters(), lr=0.0001, momentum=0.9)

In [258]:
densenet.to(device)
net.to(device)
f.to(device)
densenet.eval()
net.eval()
epochs = 20
accs_ms = list()
for epoch in range(epochs):
    for i, data in enumerate(trainloader,0):
        
        with torch.no_grad():
            inputs_audio = data['ms_spot'].unsqueeze(1).permute(0,1,2,3).to(device)
            inputs_visual = data['resnet_spot'].unsqueeze(1).permute(0,1,3,2).to(device)
            inputs_audio[inputs_audio.isnan()] = 0.0
            outputs_audio = densenet(inputs_audio)
            outputs_visual = net(inputs_visual)
            
        target = data['label'].to(device)
        
        f.train()
        optimizer.zero_grad()


        # forward + backward + optimize
        outputs = f(outputs_audio,outputs_visual)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            running_loss += loss.item()
            # print statistics


    # calculate accuracy
    with torch.no_grad():
        f.eval()
        res = torch.zeros((3,3))
        
        with torch.no_grad():
            for i, data in enumerate(validloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs_audio = data['ms_spot'].unsqueeze(1).permute(0,1,2,3).to(device)
                inputs_visual = data['resnet_spot'].unsqueeze(1).permute(0,1,3,2).to(device)
                inputs_audio[inputs_audio.isnan()] = 0.0
                outputs_audio = densenet(inputs_audio)
                outputs_visual = net(inputs_visual)

                # forward + backward + optimize
                outputs = f(outputs_audio,outputs_visual)

                preds = torch.argmax(outputs,dim=1)

                for p,gt in zip(preds,label):
                    res[int(p),int(gt)] += 1




        N_total = res.sum()
        N_correct = res.diag().sum()

        acc = N_correct / N_total
        accs_ms.append(acc)
        print(f"Epoch : {epoch}, Accuracy : {acc}")


print('Finished Training')


torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

...

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
Epoch : 17, Accuracy : 0.37308868765830994
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size(

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size([12, 256])
torch.Size

In [ ]:
resnet18 = models.resnet18(pretrained=True)